# Lecture 2 - Case study
### Assume that you are in charge of evacuating a set of resources (1000 cargoes, 800 people, and 20 wheelchair users) from a small town due to a coming flood. At your disposal you may hire trucks, big taxis, and buses, each associated with a cost of € 10,000, € 1,100, and € 10,000, respectively. You know that trucks can carry 100 units of cargo and 1  person, taxis can transport 1 unit of cargo, 4 people, 2 wheelchair users, buses can carry 10 units of cargo, 50 people, 1  wheelchair user. How many taxis, buses and trucks would you use?

In [1]:
#Resources and vehicles
vehicles = {'Truck', 'Taxi', 'Bus'}
resources = {'Cargo' , 'Person' , 'Wheelchair'}

## Given the problem description, we can arrange the input data in  _dictionaries_ before building our model.

In [2]:
##Nedeed capacity
nedeedCapacity = {'Cargo': 1000, 'Person': 800, 'Wheelchair': 20}

##Capacity for each vehicle and resource
capacity = { ('Truck', 'Cargo'): 100 , ('Truck', 'Person'): 1, ('Truck', 'Wheelchair'): 0, 
            ('Taxi', 'Cargo'): 1 , ('Taxi', 'Person'): 4, ('Taxi', 'Wheelchair'): 2,
            ('Bus', 'Cargo'): 10 , ('Bus', 'Person'): 50, ('Bus', 'Wheelchair'): 1
           }
cost = {'Truck' :10000 , 'Taxi':1100 , 'Bus':10000 }

In [3]:
## Import pyomo
import pyomo.environ as pe
import pyomo.opt as po

In [4]:
## Init model and optimizer 
model = pe.ConcreteModel()
solver = po.SolverFactory('gurobi')

In [5]:
## Sets of indices
model.V = pe.Set(initialize = vehicles, ordered = False)
model.R = pe.Set(initialize = resources, ordered = False)

In [6]:
## Variables
model.x = pe.Var(model.V, domain = pe.NonNegativeIntegers)

In [7]:
##Objective
obj_expr = sum(model.x[v] * cost[v] for v in model.V)
model.obj = pe.Objective(expr = obj_expr, sense=pe.minimize)

### In this case we have a constraint for each resource. We will use a _constraintlist_.

In [8]:
model.resource_capacity = pe.ConstraintList()
for r in model.R:
    con_expr = sum(model.x[v] * capacity[v,r] for v in model.V)>= nedeedCapacity[r]
    print(con_expr) ## Some printing to check
    model.resource_capacity.add(con_expr)
## we added all the constraints to our constraints list attached to the model.

800  <=  x[Truck] + 4*x[Taxi] + 50*x[Bus]
1000  <=  100*x[Truck] + x[Taxi] + 10*x[Bus]
20  <=  2*x[Taxi] + x[Bus]


In [9]:
## The model is complete and can be solved
result = solver.solve(model, tee= True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-14
Read LP format model from file C:\Users\timva\AppData\Local\Temp\tmpj3wjcrsw.pyomo.lp
Reading time = 0.00 seconds
x4: 4 rows, 4 columns, 9 nonzeros
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4 rows, 4 columns and 9 nonzeros
Model fingerprint: 0x8851862e
Variable types: 1 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+03, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 8 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)
Found heuristic solution: objective 1000000.00000

Root relaxation: objective 2.430936e+05, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objec

In [10]:
print(result.solver.status)
print(result.solver.termination_condition)

ok
optimal


In [11]:
print("Objective = "+str(pe.value(model.obj)))
for v in model.V: 
    if pe.value(model.x[v] > 0):
        print(str(model.x[v]) + " = " + str(pe.value(model.x[v])))

Objective = 252100.0
x[Truck] = 9.0
x[Taxi] = 11.0
x[Bus] = 15.0
